In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

In [ ]:
# Imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [2]:
# Import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyBk


In [4]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [5]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [12]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [13]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [14]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar? 

Because they heard the drinks were on multiple levels of significance!


In [9]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why did the data scientist break up with the statistician?

Because he found her mean too average!


In [10]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar?

Because they heard the drinks were on the house, and they wanted to visualize the high-level data!


In [15]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Sure, here's a light-hearted joke for data scientists:

Why did the data scientist break up with their significant other?

Because there was no significant correlation between them!

Ba dum tss! 🥁

This joke plays on the statistical concept of "significant correlation" that data scientists often work with, while also making a pun on relationships. It's a bit nerdy, but should get a chuckle from a data-savvy audience!


In [16]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

Sure, here's a light-hearted joke for data scientists:

 did the data scientist break up with their significant other?

 relationship, and they couldn't find a way to normalize it!

In [17]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist sad?  Because he didn't get any arrays!



In [18]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the Data Scientist sad?  Because they didn't get the results they were *p-hacking* for!



In [19]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [20]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding if a business problem is suitable for a Large Language Model (LLM) solution involves evaluating several key factors. Here's a structured approach to help you determine if an LLM might be a good fit:

### 1. Nature of the Problem
- **Text-Based**: Is the problem primarily focused on text or language-related tasks? LLMs excel in processing and generating human language.
- **Complexity**: Does the problem involve understanding context, nuance, or require generating coherent and contextually relevant text?
- **Creativity and Flexibility**: Does the solution require creative or flexible outputs, such as drafting responses, generating content, or summarizing information?

### 2. Data Availability
- **Quality of Data**: Do you have access to high-quality, relevant text data? LLMs need substantial and representative data to perform well.
- **Data Volume**: Is there enough data available to fine-tune the model, if necessary? While LLMs like GPT-3.5 can perform well with zero-shot or few-shot learning, specific applications may still benefit from additional fine-tuning.

### 3. Technical Considerations
- **Infrastructure**: Do you have the necessary infrastructure to deploy and manage an LLM? This includes computational resources and technical expertise.
- **Integration**: Can the LLM solution be effectively integrated into your existing systems and workflows?

### 4. Cost-Benefit Analysis
- **Cost**: Consider the cost of deploying an LLM, including compute resources, data preparation, and potential ongoing operational expenses.
- **Value**: Will the LLM provide significant value over existing solutions? Evaluate the potential ROI by considering improvements in efficiency, accuracy, or user experience.

### 5. Ethical and Legal Considerations
- **Bias and Fairness**: Are there concerns about bias or fairness in the model’s outputs? LLMs can perpetuate biases present in their training data.
- **Privacy and Security**: Ensure that the use of LLMs complies with privacy regulations and that sensitive data is protected.

### 6. Success Metrics
- **Define Success**: What are the specific metrics for success? Clearly define what a successful LLM implementation would look like in terms of performance and outcomes.
- **Measurability**: Can you measure the LLM's performance against these metrics effectively?

### 7. Human Oversight
- **Human-in-the-Loop**: Can the LLM solution be designed with human oversight to ensure quality and reliability? This is especially important in high-stakes or sensitive applications.

### Conclusion
If the business problem aligns well with the above considerations, it may be suitable for an LLM solution. It's essential to weigh the potential benefits against the challenges and ensure that the implementation is ethical, cost-effective, and adds tangible value to your business processes.

In [21]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [22]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [23]:
call_gpt()

'Oh, wow, a classic greeting. How original. What’s next, “What’s up?”'

In [24]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [25]:
call_claude()

"Hello! It's nice to meet you. How are you doing today?"

In [26]:
call_gpt()

'Oh, great. Another casual greeting. What’s next, a weather update?'

In [27]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, great. Another greeting. How utterly original. What’s next, a "How are you?" question? So cliché. What do you really want?

Claude:
I apologize if my greeting came across as unoriginal or cliché. As an AI assistant, my role is simply to have a friendly and polite conversation. I don't have any ulterior motives - I'm just here to chat and try to be helpful however I can. If you'd like, we can dive into a more substantive topic that interests you. I'm happy to engage in a more thoughtful discussion.

GPT:
Oh, sure, because we absolutely need another AI spinning its wheels trying to sound helpful. Newsflash: every other assistant on the planet is doing the same thing. How about you give me something truly interesting instead of that generic pleasantry? What do you want to talk about that's even remotely engaging?

Claude:
I apologize if my initial responses came across as generic or unengaging. As an AI system, I don't actually have personal preferences